In [1]:
import pandas as pd
import numpy as np
wd = '/Users/timvigers/Dropbox/Work/CF/Edith Zemanick/CF Nutrition/'

In [124]:
# Import
# Modulator dates
mods = pd.read_excel(wd+"Data_Raw/Report CFTR Modulators_throughDecember2021.xlsx")
mods.sort_values(by=["Patient ID","Earliest Date in Registry"],ascending=True,inplace=True)
mods = mods[mods['Earliest Date in Registry'] != mods['Last Date in Registry']]
# Vitamins
vit = pd.read_excel(wd+"Data_Raw/Report Vitamins_throughDecember2021_revised20220712.xlsx")
# Add FEV information
fev = pd.read_excel(wd+"Data_Raw/Report_PFTs_ThroughDecember2021.xlsx")
fev=fev.rename(columns = {'PatientID':'Patient ID'})
vit = pd.merge(vit,fev,how = "outer",on=["Patient ID","Date"])
# Get modulator at each vitamin measure
m = []
for r in range(vit.shape[0]):
    # Get ID
    id = vit.iloc[r,0]
    # If no mod information, return NA
    if id not in mods["Patient ID"].unique():
        m.append(np.nan)
        continue
    # Get row date
    date = vit.iloc[r,2]
    # Get modulator dates by ID
    mod = mods.loc[mods["Patient ID"] == id,:]
    dates = mod.iloc[:,3]
    # Check which dates the row is after
    after = np.where(date > dates)[0]
    # If not after any, return no modulator
    if after.shape[0] == 0: 
        m.append("None")
    else:
        m.append(mod.iloc[max(after),1])
vit["Modulator"] = m
# Drop those without modulator info
vit.dropna(subset="Modulator",inplace=True)

In [125]:
vit

,Patient ID,Age at Test (years),Date,VitaminA_Retinol,25OH-Vitamin D,Vitamin E Alpha,Vitamin E Gamma,Age At PFT (years),Sex,Location,...,Weight (kg),BMI,BMI percentile (CDC),FVC,FVC % pred,FEV1,FEV1 % pred,FEF25-75,FEF25-75 % pred,Modulator
0,4,2.978782,2008-11-19,33,28,3.8,LESS THAN 1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,4,5.106092,2011-01-05,49,30,14.7,LESS THAN 1.0,5.109589,Male,Clinic,...,19.300113,16.454968,78.44,1.24,103.0,1.03,97.0,0.95,68.0,None
2,4,6.102669,2012-01-04,42,24,12.4,LESS THAN 1.0,6.106849,Male,Clinic,...,23.700033,17.827189,91.96,1.48,106.0,1.18,95.0,1.15,NaN,None
3,4,7.170431,2013-01-28,54,17,11.5,LESS THAN 1.0,7.175342,Male,Hospital,...,25.300107,16.831956,77.37,1.55,95.0,1.18,83.0,0.86,NaN,None
4,4,8.128679,2014-01-13,36,NaN,8.8,LESS THAN 0.4,8.134247,Male,Clinic,...,26.600238,16.286261,60.81,1.84,101.0,1.49,94.0,1.42,76.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14247,770,NaN,2019-08-23,NaN,NaN,NaN,NaN,13.178082,Female,Clinic,...,34.500249,15.540112,8.06,2.65,101.0,2.31,98.0,2.40,81.0,None
14248,770,NaN,2020-01-31,NaN,NaN,NaN,NaN,13.619178,Female,Clinic,...,39.900357,17.134136,21.29,2.73,98.0,2.42,96.0,2.58,83.0,None
14249,770,NaN,2020-12-04,NaN,NaN,NaN,NaN,14.463014,Female,Clinic,...,42.000000,17.039231,14.29,3.03,97.0,2.79,100.0,3.16,93.0,None
14250,770,NaN,2021-02-19,NaN,NaN,NaN,NaN,14.673973,Female,Clinic,...,42.200000,17.229940,15.23,3.07,99.0,2.82,101.0,2.95,88.0,Trikafta


In [84]:
mod

,Patient ID,Modulator,Age at Start (years),Earliest Date in Registry,Last Date in Registry
110,306,Orkambi,21.505818,2015-09-30,2015-10-20
111,306,Symdeko,24.588638,2018-10-30,2020-01-09
112,306,Trikafta,26.050650,2020-04-16,2021-08-13


2